# Object Detection and Segmentation

Other than the typical classification related usecases, some of the main other vision use cases are object detection and segmentation. Object detection is identifying the objects in a image. Segmentation means separating out image elements. This can be instance level separation or whole image separation. These 2 types are called instance segmentation and whole-scene semantic segmentation.

## Object detection

Unlike image classification, where we only try to detect whether there's an object, here we try to locate that object in the image. But as you we can already see, object classification already need to locate the object before classification. Problem is those algorithms does not return the object location information. So to make use of that, easiest solution is to predict bounding boxes arorund the detected object. That's in fact the idea behind 'YOLO' approach. 

Unfortunately using typical network architectures does not yield the best result for object detection. Therefore researchers introduced a new architecture called `Feature Pyramid Networks or FPNs` and illustrated their usage with RetinaNet.

---

### **YOLO (You Only Look Once)**

This is one of the simplest object detection architectures with fastest prediction times. Due to this reason it is used in many real time applications. Also this has now been evolved in to many other versions like YOLO v2 - v5 as well with slight differences to the architecture for better prediction result and performance improvements.

Basic idea behind YOLO is that it divides an image to NxM cells and tries to predict a bounding box for an object that may be in those cells. Bounding box may not necessarilly be same size as the initial cell. But predicted box center should be inside of the considering cell. 

<center><image src="./imgs/8.png" width="400px"/></center>

Then for each of these grid cell, YOLO will output below values.

    x: center x of the detected object in that cell (tanh activation)
    y: center y of the detected object in that cell (tanh activation)
    w: width of the detected object as ratio (sigmoid activation)
    h: width of the detected object as ratio (sigmoid activation)
    c: confidence for the detected object
    class: this is a softmax for the classes we train our model

<center><image src="./imgs/9.png" width="600px"/></center>

Note in the above image feature map is for a single data input (For a single image). So single vertical column depicts the outputs related to single grid cell.

As you can see due to this reason, it is important to make sure that final feature map have correct dimentions so that we can get the desired representation of output. One way to achieve this is carefully design the ML network to get the required dimensions. But typically this is cumbersome to achieve. Instead authors suggested to flatten the 3D feature map and then send it through a fully connected layer with size (grid_w x grid_h x (5 + numclasses)) and then reshape it to our requirement. Apparently this yields better results compared to directly using the feature map.

To build the loss function for this we need training data with ground truth boxes and their classes. So then we can compare our output predicted boxes against the ground truth boxes. Also loss function need to consider classification results as well. This is straight forward if a grid cell predicts a single box. But this is not the case in practice. Because most of the time there may be cases some objects may space across multiple grid cells making them overlap. In these cases having one prediction box per cell is not enough. To solve this problem, in YOLO architecture there's a parameter to define number of bounding boxes per a grid cell. This means in the output there will be more x,y,w,h,c values. Check the below image.

<center><image src="./imgs/10.png" width="400px"/></center>

So with these new additions, comparing the prediction with ground truth becomes bit more complex. In fact, instead of just directly comparing each prediction with the ground truth, YOLO uses a metric called 'IOU' or Intersection over Union between all the ground truth boxes and all predicted boxes within a grid cell. Then it select the pairing with highest IOU as the correct prediction.

<center><image src="./imgs/11.png" width="400px"/></center>

With this in place, Total loss per a sample would be a combination of below values.

1. **Object Presence Loss:**
        Each cell that has a ground truth box, compute (1-C)^2. (Loss for not correctly predicting the object presence).
2. **Object Absence Loss:**
        Each cell that does not have a ground truth box computes C^2. (Loss for predicting a object which is not available)
3. **Object classification Loss:**
        Typical cross entropy loss (Loss for incorrectly classifying the detected object).
4. **Bounding Box Loss**
        Loss for bad bounding box dimension prediction.

Then YOLO combine all above losses with different weighting factors to optimize the model for object detection.

One of the  main limitation of the YOLO architecture is that it can only predict single class per grid cell(can select multiple boxes per grid cells though). So will not work if there are multilple items in single grid cell.

Another issue is grid itself. This grid cell separation makes the model not invariant to scale. So unless we carefully tune the model, it will not be able to identify small objects.

Also YOLO tends to locate objects with low precision. This may be because YOLO tries to identify the bounding box from the last feature map of the network. (Usually at such level, spacial details of the original image tend to not be in the last feature map)

---

### **RetinaNet**

Compared to the YOLOv1, this has several new concepts used in its architecture and in its loss calculation. Their architecture design include feature pyramid networks which combine information extracted in multiple scales as well. 

#### **Feature Pyramid Maps**


In the Yolo architecture, detection part only used the last high level feature maps that comes after all the processings of the convolutional layers. This causes the model to be less accurate in its localization capability as the high level features have less spacial resolution by design. 

To circumvent this problem, researchers looked in to a way to combine early level feature maps into detection head inputs hoping that would help to improve the localization capabilities. Thus feature pyramid networks were born. Check the below diagram.

<center><image src="./imgs/12.png" width="500px"/></center>

As we can see in Feature pyramid networks high level feature maps get upsampled and transfered back to low level. Then such combined feature maps will be used in detection tasks. 

<center><image src="./imgs/13.png" width="500px"/></center>

In the FPN, high level feature maps get upsampled and then get added up to the low level feature maps. This helps the spacial features to get preserved while having fine tuned fearures. Then such combined feature maps will get send to the detection heads to produce the detection box and classification. The point to note is that, channel size of inputs to the detection head. Since all these inputs get fed into same (shared) detection head it is important to have same depth size. (Other dimensions can be managed because of the convolutional operations)

Also FPN does not depend on the convolutional network it is being used as long as it can access the intermediate feature maps. So it can be used with any major networks like Resnets or Efficient nets. 


#### **Anchor Boxes**

